In [56]:
@time include("setup.jl")

Total number of nodes on fault = 121
Average node spacing = 400.0
dt = 0.04486312014474891
Setup Complete
  0.849824 seconds (481.10 k allocations: 28.290 MiB, 0.53% gc time)


In [ ]:
# Plot friction parameters
using Gadfly

fricPlot = plot(layer(x = (cca-ccb).*1e3 + 10, y = FltX,Theme(default_color=colorant"red"), Geom.point),
                layer(x = Seff/1e6, y = FltX, Theme(default_color=colorant"deepskyblue"), Geom.point),
                layer(x = tauo/1e6, y = FltX, Theme(default_color=colorant"green"), Geom.point),
                Guide.xlabel("Scaled (a-b)/ Stress value"),
                Guide.ylabel("Depth (m)"),
                Guide.title("Rate and state friction/Stress"),
                Coord.Cartesian(ymin=-24))

In [57]:
@time include("timesolver.jl")

LoadError: LoadError: [91mNR SEARCH FAILED![39m
while loading /Volumes/GoogleDrive/My Drive/Work/Code/JuliaSEMCycle/timesolver.jl, in expression starting on line 75

In [51]:
wloc = W[:,:,1]; k = 1; j = 1
sum( coefint1*H[k,:].*(wloc[:,j].*Ht[:,k]) + coefint2*(wloc[k,:].*H[j,:])'.*Ht[:,j] )

3.611797470450001e10

In [60]:
Vf

121×1 Array{Float64,2}:
   1.10984e-9 
   1.10984e-9 
   1.10984e-9 
   1.10984e-9 
   1.10984e-9 
   1.10984e-9 
   1.10984e-9 
   1.10984e-9 
   1.10984e-9 
   1.10984e-9 
   1.10984e-9 
   1.10984e-9 
   1.10984e-9 
   ⋮          
   0.0127053  
   0.0127109  
   0.0127147  
   0.0127318  
   0.0126983  
   0.0128332  
   0.00738499 
   0.0241738  
   0.000160063
   0.545583   
  -6.25424    
 159.314      

In [42]:
coefint2

1.0666666666666667